In [85]:
""" 
--- Day 3: Gear Ratios ---

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water 
source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting 
anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which 
one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part 
is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of 
numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, 
is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) 
and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. 
What is the sum of all of the part numbers in the engine schematic?

"""

from string import punctuation
import re

P = punctuation.replace(".", "")

def get_part(s: str, found: re.Match) -> int:
    row_len = len(s[0])    
    num = found.group()
    start = found.start()

    y, x = divmod(start, row_len) 
    x_off = max(0, x - 1)
    y_range = range(max(0, y - 1), min(y + 2, len(s)))

    segment = "\n".join(s[iy][x_off: x + len(num) + 1] for iy in y_range)
    return any(c in P for c in segment) * int(num)

def sum_of_parts(s: str) -> int:
    nums = re.finditer(r"\d+", s.replace("\n", ""))
    return sum(get_part(s.split("\n"), num) for num in nums)
    
with open("input.txt") as f:
    print(sum_of_parts(f.read()))

#sum is 551094

551094


In [122]:
"""
--- Part Two ---

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the 
closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone 
labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding 
a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You 
exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is 
adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out 
which gear needs to be replaced.

Consider the same engine schematic again:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear 
ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a 
gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

"""

import re
from typing import List


class Number:
    def __init__(self, number: re.Match, row_len: int):
        self.num = number.group()
        self.y, self.x = divmod(number.start(), row_len)
        self.x_range = range(self.x, self.x + len(self.num))        
        
    def in_range(self, gear_y: int, gear_x: int) -> bool:
        return gear_y == self.y and gear_x in self.x_range


def find_gears(s: str) -> int:
    size = len(s.split("\n")[0])
    gears = re.finditer(r"\*", s.replace("\n", ""))
    total = 0
    
    for gear in gears:
        neighbors = check_around(s, size, *divmod(gear.start(), size))
        if len(neighbors) == 2:
            total += neighbors[0] * neighbors[1]
    return total
    
def check_around(s: str, size: int, gear_y: int, gear_x: int) -> int:
    nums = get_nums(s, size)
    parts = set()
    
    for num in nums:
        for offsety in range(-1, 2):
            for offsetx in range(-1, 2):
                if num.in_range(gear_y + offsety, gear_x + offsetx):
                    parts.add(num)
    return [int(num.num) for num in parts]

def get_nums(s: str, row_len: int) -> List[Number]:
    nums = re.finditer(r"\d+", s.replace("\n", ""))
    nums = [Number(num, row_len) for num in nums]
    return nums

    
with open("input.txt") as f:
    print(find_gears(f.read()))

#sum is 80179647

80179647
